In [2]:
import numpy as np
import cv2 as cv2
from numpy.core import asarray, zeros, swapaxes, take
from numpy.core.multiarray import normalize_axis_index
from numpy.fft import _pocketfft_internal as pfi
from scipy.optimize import curve_fit
import pandas as pd
import os
import time

from matplotlib import pyplot as plt
from tqdm import tqdm

In [3]:
# Image storage
class matchimg:
    '''
    img_old: name of the reference image (older)
    img_new: name of the secondary image (newer)
    fold: address of the images
    '''
    def __init__(self,fold_urf,img_old_urf,img_new_urf):
        self.img_old = img_old_urf
        self.img_new = img_new_urf
        self.fold = fold_urf

    # read two images
    def read_img(self):
        self.img_old = cv2.imread(self.fold+self.img_old, cv2.IMREAD_GRAYSCALE)
        self.img_new = cv2.imread(self.fold+self.img_new, cv2.IMREAD_GRAYSCALE)

# Fourier NCC
class img2vxy_fourier:
    '''
    imas: two images (class)
    delta_x/y: temporary displacement x/y
    lengthxy: size of results
    ker_size: size of search window
    win_size: size of search range
    startxy: topleft point of the result
    isprocessed: temporary data
    foldurf: target address of results
    finish/err_name_urf: temporary data
    '''
    def __init__(self,matchimg,ker_size,win_size,startxy,lengthxy,
                 foldurf,finish_name_urf,err_name_urf):
        self.imgs = matchimg
        self.delta_x = np.zeros((lengthxy[0],lengthxy[1]))
        self.delta_y = np.zeros((lengthxy[0],lengthxy[1]))
        self.lengthxy = lengthxy
        self.ker_size = ker_size
        self.win_size = win_size
        self.startxy = startxy
        self.isprocessed = np.zeros_like(self.delta_x)

        self.foldurf = foldurf
        self.finish_name_urf = finish_name_urf
        self.err_name_urf = err_name_urf

    #..........................................................
    '''
    Fourier algorithm of NCC
    Reference: https://blog.csdn.net/djq_313/article/details/131178037
    '''
    def raw_fft(self,a, n, axis, is_real, is_forward, inv_norm):
        axis = normalize_axis_index(axis, a.ndim)
        if n is None:
            n = a.shape[axis]

        fct = 1/inv_norm

        if a.shape[axis] != n:
            s = list(a.shape)
            index = [slice(None)]*len(s)
            if s[axis] > n:
                index[axis] = slice(0, n)
                a = a[tuple(index)]
            else:
                index[axis] = slice(0, s[axis])
                s[axis] = n
                z = zeros(s, a.dtype.char)
                z[tuple(index)] = a
                a = z

        if axis == a.ndim-1:
            r = pfi.execute(a, is_real, is_forward, fct)
        else:
            a = swapaxes(a, axis, -1)
            r = pfi.execute(a, is_real, is_forward, fct)
            r = swapaxes(r, axis, -1)
        return r
    def fft(self,a, n=None, axis=-1):
        a = asarray(a)
        if n is None:
            n = a.shape[axis]
        inv_norm = 1
        output = self.raw_fft(a, n, axis, False, True, inv_norm)
        return output
    def ifft(self,a, n=None, axis=-1):
        a = asarray(a)
        if n is None:
            n = a.shape[axis]
        inv_norm = n
        output = self.raw_fft(a, n, axis, False, False, inv_norm)
        return output
    def cook_nd_args(self,a, shape=None, axes=None, invreal=0):
        if shape is None:
            shapeless = 1
            if axes is None:
                shape = list(a.shape)
            else:
                shape = take(a.shape, axes)
        else:
            shapeless = 0
        shape = list(shape)
        if axes is None:
            axes = list(range(-len(shape), 0))
        if invreal and shapeless:
            shape[-1] = (a.shape[axes[-1]] - 1) * 2
        return shape, axes
    def raw_fftnd(self,a, shape=None, axes=None, function=fft):
        a = asarray(a)
        shape, axes = self.cook_nd_args(a, shape, axes)
        itl = list(range(len(axes)))
        itl.reverse()
        for ii in itl:
            a = function(a, n=shape[ii], axis=axes[ii])
        return a
    def imagefft2(self,a, shape=None, axes=(-2, -1)):
        return self.raw_fftnd(a, shape, axes, self.fft)
    def imageifft2(self,a, shape=None, axes=(-2, -1)):
        return self.raw_fftnd(a, shape, axes, self.ifft)
    #..........................................................

    # Use Fourier to get CI matrix
    def fft2CI(self,ker,win):
        win_height, win_width = win.shape
        #Fourier function
        t_fft = self.imagefft2(ker, shape=(win_height, win_width))
        o_fft = self.imagefft2(win)
        ci = self.imageifft2(np.multiply(o_fft, t_fft.conj()) / np.abs(np.multiply(o_fft, t_fft.conj())))
        return ci.real

    # Find the first n maximum number
    def top_n_indices_2d(self,arr, n):
        flat_indices = np.argpartition(arr.ravel(), -n)[-n:]
        indices = np.unravel_index(flat_indices, arr.shape)
        values = arr[indices]
        return indices[0],indices[1],values

    # Judging whether the matching is successful by the relationship between the first n maximum values and coefficients
    def is_matchpoint(self,ci,n):
        index_x,index_y,index_ci = self.top_n_indices_2d(ci,n)
        # cimean = np.mean(ci)
        # cistd = np.std(ci)
        # index_ci = (index_ci-cimean)/cistd
        ci_max_idx = np.where(index_ci == np.max(index_ci))[0][0]

        degree = 0  #Evaluation criteria: degree of deviation
        for ii in range(n):
            if ii==ci_max_idx:
                continue
            if (index_x[ii]-index_x[ci_max_idx])**2 + (index_y[ii]-index_y[ci_max_idx])**2<=5:
                continue
            degree += 1/abs(index_ci[ii]-index_ci[ci_max_idx])
        return degree/n

    # Find the index where the maximum value is located
    def findmax(self,ci):
        def parabola_fit(x, a, b, c, d, e, f):
            return (a*x[0]**2 + b*x[1]**2 + c*x[0]*x[1] + d*x[0] + e*x[1] + f).ravel()

        result = np.where(ci == np.amax(ci))
        result = np.array(list(zip(result[0], result[1]))[0])
        ci_size = ci.shape
        if np.any(np.logical_or(result>=ci_size[0]-2,result<=1)):
            return np.nan,np.nan
        new_ci = ci[result[0]-2:result[0]+3,result[1]-2:result[1]+3]*100
        para_xy = np.indices([5,5])-2
        para_z = new_ci.flatten()
        para_popt, _ = curve_fit(parabola_fit, para_xy, para_z)

        para_A = np.array([[2*para_popt[0],para_popt[2]],[para_popt[2],2*para_popt[1]]])
        para_b = np.array([-para_popt[3],-para_popt[4]])
        para_solvexy = np.linalg.solve(para_A,para_b)

        if para_solvexy[0]**2 + para_solvexy[1]**2>8:
            return np.nan,np.nan
        return result[1]+para_solvexy[1],result[0]+para_solvexy[0]

    # Store files
    def auto_middle_save(self,name_urf):
        df = pd.DataFrame(self.delta_x)
        df.to_csv(self.foldurf+name_urf[0]+'.csv', index=False)
        df = pd.DataFrame(self.delta_y)
        df.to_csv(self.foldurf+name_urf[1]+'.csv', index=False)
        df = pd.DataFrame(self.isprocessed)
        df.to_csv(self.foldurf+name_urf[2]+'.csv', index=False)

    # Registration
    def matchprocess(self):
        start_time = time.time()
        for ii in tqdm(range(self.lengthxy[0])):
            for jj in range(self.lengthxy[1]):
                if self.isprocessed[ii,jj]:
                    continue
                imgker = self.imgs.img_old[self.startxy[0]+ii-self.ker_size//2:self.startxy[0]+ii+self.ker_size//2,
                                           self.startxy[1]+jj-self.ker_size//2:self.startxy[1]+jj+self.ker_size//2]
                imgwin = self.imgs.img_new[self.startxy[0]+ii-self.win_size//2:self.startxy[0]+ii+self.win_size//2,
                                           self.startxy[1]+jj-self.win_size//2:self.startxy[1]+jj+self.win_size//2]
                ci = self.fft2CI(imgker,imgwin)
                if np.isnan(ci[0,0]):
                    self.delta_x[ii,jj],self.delta_y[ii,jj] = np.nan,np.nan
                    # print(f'{self.startxy[0]+ii} , {self.startxy[1]+jj} is missing')
                    self.isprocessed[ii,jj] = 2
                    continue
                degree = self.is_matchpoint(ci,5)  #first n maximum number
                if degree > 150:  #threshold of the degree
                    self.delta_x[ii,jj],self.delta_y[ii,jj] = np.nan,np.nan
                    self.isprocessed[ii,jj] = 2
                    continue
                tmpdx,tmpdy = self.findmax(ci)
                self.delta_x[ii,jj],self.delta_y[ii,jj] = tmpdx,tmpdy
                if np.isnan(tmpdx):
                    self.isprocessed[ii,jj] = 2
                else:
                    self.isprocessed[ii,jj] = 1

            loop_time = time.time()-start_time
            if loop_time>3600:
                self.auto_middle_save(self.err_name_urf)
                print('Automatic backup completed every hour!')
                start_time = time.time()

    # complete process for Fourier NCC
    def completeprocess(self,is_after_err=False,past_arr_urf=None):
        if is_after_err:
            df = pd.read_csv(self.foldurf+past_arr_urf[0]+'.csv')
            self.delta_x = np.array(df)

            if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
                print('The unfinished task does not match the object! Please check the object parameters!')
                print(f'Size of the unfinished task: {self.delta_x.shape}')
                print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
                return None

            df = pd.read_csv(self.foldurf+past_arr_urf[1]+'.csv')
            self.delta_y = np.array(df)

            df = pd.read_csv(self.foldurf+past_arr_urf[2]+'.csv')
            self.isprocessed = np.array(df)

        try:
            self.matchprocess()
        except KeyboardInterrupt:
            print('Keyboard Interrupt!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        except ZeroDivisionError:
            print('Zero Division Error!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        except IndexError:
            print('Index Error!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        else:
            print('Finished!')
            self.auto_middle_save(self.finish_name_urf)
            print('Storage finished.')
            return 1

    # Read completed files
    def read_finished_vxy(self,name_urf):
        df = pd.read_csv(self.foldurf+name_urf[0]+'.csv')
        self.delta_x = np.array(df)

        if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
            print('The completed task does not match the object! Please check the object parameters!')
            print(f'Size of the completed task: {self.delta_x.shape}')
            print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
            return None

        df = pd.read_csv(self.foldurf+name_urf[1]+'.csv')
        self.delta_y = np.array(df)

        df = pd.read_csv(self.foldurf+name_urf[2]+'.csv')
        self.isprocessed = np.array(df)

        return True

# Fill in gaps
class img2vxy_equal:
    '''
    imas: two images (class)
    delta_x/y: temporary displacement x/y
    lengthxy: size of results
    ker_size: size of search window
    win_size: size of search range
    startxy: topleft point of the result
    isprocessed: temporary data
    foldurf: target address of results
    finish/err_name_urf: temporary data
    '''
    def __init__(self,matchimg,ker_size,win_size,startxy,lengthxy,
                 foldurf,finish_name_urf,err_name_urf):
        self.imgs = matchimg

        self.delta_x = np.zeros((lengthxy[0],lengthxy[1]))
        self.delta_y = np.zeros((lengthxy[0],lengthxy[1]))
        self.lengthxy = lengthxy
        self.ker_size = ker_size
        self.win_size = win_size
        self.startxy = startxy
        self.isprocessed = np.zeros_like(self.delta_x)

        self.foldurf = foldurf
        self.finish_name_urf = finish_name_urf
        self.err_name_urf = err_name_urf


    # Read completed Fourier files
    def read_fourier_vxy(self,out_foldurf,name_urf,is_used_fourier=False,fourier=None):
        # Use the results obtained from exterior Fourier
        if not is_used_fourier:
            df = pd.read_csv(out_foldurf+name_urf[0]+'.csv')
            self.delta_x = np.array(df)-self.win_size//2+self.ker_size//2
            if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
                print('The Fourier task does not match this object! Please check the object parameters!')
                print(f'Size of Fourier task: {self.delta_x.shape}')
                print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
                return None

            df = pd.read_csv(out_foldurf+name_urf[1]+'.csv')
            self.delta_y = np.array(df)-self.win_size//2+self.ker_size//2

            df = pd.read_csv(out_foldurf+name_urf[2]+'.csv')
            self.isprocessed = np.array(df)
            print('Importing Fourier processing file completed!')
        # Use the results obtained from inner Fourier
        else:
            self.delta_x = fourier.delta_x-self.win_size//2+self.ker_size//2
            if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
                print('The Fourier task does not match this object! Please check the object parameters!')
                print(f'Size of Fourier task: {self.delta_x.shape}')
                print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
                return None
            self.delta_y = fourier.delta_y-self.win_size//2+self.ker_size//2
            self.isprocessed = fourier.isprocessed
            print('Importing Fourier processing memory completed!')

    #Clear abnormal data and retain data within the range
    def save_range_data(self,dxrange,dyrange):
        nanidx = np.where(np.logical_or(np.logical_or(self.delta_x<dxrange[0],self.delta_x>dxrange[1]),
                                       np.logical_or(self.delta_y<dyrange[0],self.delta_y>dyrange[1])))
        self.delta_x[nanidx] = np.nan
        self.delta_y[nanidx] = np.nan
        self.isprocessed[nanidx] = 2

    # Find the best matching location based on the existing results around the grid
    def choose_nearest_distance(self,ii,jj):
        radius = 1
        tmpstate = 0
        maxiter = 0

        while not tmpstate:
            maxiter+=1
            tmp_isprocessed = self.isprocessed[max(0,ii-radius):min(self.isprocessed.shape[0],ii+radius+1),
                                               max(0,jj-radius):min(self.isprocessed.shape[1],jj+radius+1)]
            tmp_dx = self.delta_x[max(0,ii-radius):min(self.isprocessed.shape[0],ii+radius+1),
                                  max(0,jj-radius):min(self.isprocessed.shape[1],jj+radius+1)]
            if 2*tmp_isprocessed.shape[0]*tmp_isprocessed.shape[1] - np.sum(tmp_isprocessed) > 5 and (not np.isnan(tmp_dx).all()):
                tmpstate = 1
            else:
                radius += 1
            if maxiter>1000:
                return 0,0

        tmp_isprocessed = self.isprocessed[max(0,ii-radius):min(self.isprocessed.shape[0],ii+radius+1),
                                           max(0,jj-radius):min(self.isprocessed.shape[1],jj+radius+1)]

        tmp_idx = np.where(tmp_isprocessed==1)

        tmp_dx = self.delta_x[max(0,ii-radius):min(self.isprocessed.shape[0],ii+radius+1),
                              max(0,jj-radius):min(self.isprocessed.shape[1],jj+radius+1)]
        tmp_dx = tmp_dx[tmp_idx[0],tmp_idx[1]]
        tmp_ave_dx = np.nanmedian(tmp_dx)

        tmp_dy = self.delta_y[max(0,ii-radius):min(self.isprocessed.shape[0],ii+radius+1),
                              max(0,jj-radius):min(self.isprocessed.shape[1],jj+radius+1)]
        tmp_dy = tmp_dy[tmp_idx[0],tmp_idx[1]]
        tmp_ave_dy = np.nanmedian(tmp_dy)

        return int(tmp_ave_dx),   int(tmp_ave_dy)

    # fill in the gaps
    def equalCI(self,img, kernel):
        ker_fft = np.fft.fft2(kernel)
        win_fft = np.fft.fft2(img)
        tmp = np.multiply(ker_fft,win_fft.conj())
        tmp = tmp/np.abs(tmp)
        tmpCImat = np.fft.ifft2(tmp)
        tmpCImat = tmpCImat.real
        CIshape = tmpCImat.shape[0]

        newCImat = np.zeros_like(tmpCImat)
        newCImat[CIshape//2:CIshape,CIshape//2:CIshape] = tmpCImat[:CIshape//2,:CIshape//2]
        newCImat[:CIshape//2,CIshape//2:CIshape] = tmpCImat[CIshape//2:CIshape,:CIshape//2]
        newCImat[CIshape//2:CIshape,:CIshape//2] = tmpCImat[:CIshape//2,CIshape//2:CIshape]
        newCImat[:CIshape//2,:CIshape//2] = tmpCImat[CIshape//2:CIshape,CIshape//2:CIshape]
        return newCImat

    # Find the first n maximum number
    def top_n_indices_2d(self,arr, n):
        flat_indices = np.argpartition(arr.ravel(), -n)[-n:]
        indices = np.unravel_index(flat_indices, arr.shape)
        values = arr[indices]
        return indices[0],indices[1],values

    # Judging whether the matching is successful by the relationship between the first n maximum values and coefficients
    def is_matchpoint(self,ci,n):
        index_x,index_y,index_ci = self.top_n_indices_2d(ci,n)
        # cimean = np.mean(ci)
        # cistd = np.std(ci)
        # index_ci = (index_ci-cimean)/cistd
        ci_max_idx = np.where(index_ci == np.max(index_ci))[0][0]

        degree = 0  #Evaluation criteria: degree of deviation
        for ii in range(n):
            if ii==ci_max_idx:
                continue
            if (index_x[ii]-index_x[ci_max_idx])**2 + (index_y[ii]-index_y[ci_max_idx])**2<=5:
                continue
            degree += 1/abs(index_ci[ii]-index_ci[ci_max_idx])
        return degree/n

    # Find the index where the maximum value is located
    def findmax(self,ci):
        def parabola_fit(x, a, b, c, d, e, f):
            return (a*x[0]**2 + b*x[1]**2 + c*x[0]*x[1] + d*x[0] + e*x[1] + f).ravel()

        result = np.where(ci == np.amax(ci))
        result = np.array(list(zip(result[0], result[1]))[0])
        ci_size = ci.shape
        if np.any(np.logical_or(result>=ci_size[0]-2,result<=1)):
            return np.nan,np.nan
        new_ci = ci[result[0]-2:result[0]+3,result[1]-2:result[1]+3]*100
        para_xy = np.indices([5,5])-2
        para_z = new_ci.flatten()
        para_popt, _ = curve_fit(parabola_fit, para_xy, para_z)

        para_A = np.array([[2*para_popt[0],para_popt[2]],[para_popt[2],2*para_popt[1]]])
        para_b = np.array([-para_popt[3],-para_popt[4]])
        para_solvexy = np.linalg.solve(para_A,para_b)

        if para_solvexy[0]**2 + para_solvexy[1]**2>8:
            return np.nan,np.nan
        return result[1]+para_solvexy[1]-self.ker_size//2,result[0]+para_solvexy[0]-self.ker_size//2


    # Registration
    def single_match(self,ii,jj):
        if self.isprocessed[ii,jj]==1:
            return None
        dy,dx = self.choose_nearest_distance(ii,jj)
        imgker = self.imgs.img_old[self.startxy[0]+ii-self.ker_size//2:self.startxy[0]+ii+self.ker_size//2,
                                   self.startxy[1]+jj-self.ker_size//2:self.startxy[1]+jj+self.ker_size//2]
        imgwin = self.imgs.img_new[self.startxy[0]+ii-self.ker_size//2+dx:self.startxy[0]+ii+self.ker_size//2+dx,
                                   self.startxy[1]+jj-self.ker_size//2+dy:self.startxy[1]+jj+self.ker_size//2+dy]
        ci = self.equalCI(imgker,imgwin)
        if np.isnan(ci[0,0]):
            return None
        degree = self.is_matchpoint(ci,5)
        if degree > 500:
            return None
        self.delta_x[ii,jj],self.delta_y[ii,jj] = self.findmax(ci)
        if np.isnan(self.delta_x[ii,jj]):
            return None
        else:
            self.delta_x[ii,jj]+=dy
            self.delta_y[ii,jj]+=dx
            self.isprocessed[ii,jj] = 1
            return True

    # Store files
    def auto_middle_save(self,name_urf):
        df = pd.DataFrame(self.delta_x)
        df.to_csv(self.foldurf+name_urf[0]+'.csv', index=False)
        df = pd.DataFrame(self.delta_y)
        df.to_csv(self.foldurf+name_urf[1]+'.csv', index=False)
        df = pd.DataFrame(self.isprocessed)
        df.to_csv(self.foldurf+name_urf[2]+'.csv', index=False)

    # fill in the gaps within the whole file
    def ranking_sequence(self,isautosave = True):
        for ii in tqdm(np.flip(np.arange(self.delta_x.shape[0]//2))):
            for jj in np.flip(np.arange(self.delta_x.shape[1]//2)):
                self.single_match(ii,jj)
        if isautosave:
            self.auto_middle_save(self.err_name_urf)
            print('Automatic backup completed!')
        for ii in tqdm(np.arange(self.delta_x.shape[0]//2,self.delta_x.shape[0])):
            for jj in np.flip(np.arange(self.delta_x.shape[1]//2)):
                self.single_match(ii,jj)
        if isautosave:
            self.auto_middle_save(self.err_name_urf)
            print('Automatic backup completed!')
        for ii in tqdm(np.flip(np.arange(self.delta_x.shape[0]//2))):
            for jj in np.arange(self.delta_x.shape[1]//2,self.delta_x.shape[1]):
                self.single_match(ii,jj)
        if isautosave:
            self.auto_middle_save(self.err_name_urf)
            print('Automatic backup completed!')
        for ii in tqdm(np.arange(self.delta_x.shape[0]//2,self.delta_x.shape[0])):
            for jj in np.arange(self.delta_x.shape[1]//2,self.delta_x.shape[1]):
                self.single_match(ii,jj)

    # complete process
    def completeprocess(self,isautosave=True,is_after_err=False,past_arr_urf=None):
        if is_after_err:
            df = pd.read_csv(self.foldurf+past_arr_urf[0]+'.csv')
            self.delta_x = np.array(df)

            if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
                print('The unfinished task does not match the object! Please check the object parameters!')
                print(f'Size of the unfinished task: {self.delta_x.shape}')
                print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
                return None

            df = pd.read_csv(self.foldurf+past_arr_urf[1]+'.csv')
            self.delta_y = np.array(df)

            df = pd.read_csv(self.foldurf+past_arr_urf[2]+'.csv')
            self.isprocessed = np.array(df)

        try:
            self.ranking_sequence(isautosave)
        except KeyboardInterrupt:
            print('Keyboard Interrupt!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        except ZeroDivisionError:
            print('Zero Division Error!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        except IndexError:
            print('Index Error!')
            self.auto_middle_save(self.err_name_urf)
            print('Temporary storage finished.')
            return 0
        else:
            print('Finished!')
            self.auto_middle_save(self.finish_name_urf)
            print('Storage finished.')
            return 1

    # Read completed files
    def read_finished_vxy(self,name_urf):
        df = pd.read_csv(self.foldurf+name_urf[0]+'.csv')
        self.delta_x = np.array(df)

        if self.delta_x.shape[0]!=self.lengthxy[0] or self.delta_x.shape[1]!= self.lengthxy[1]:
            print('The completed task does not match the object! Please check the object parameters!')
            print(f'Size of the completed task: {self.delta_x.shape}')
            print(f'Size of this object: {self.lengthxy[0]} , {self.lengthxy[1]}')
            return None

        df = pd.read_csv(self.foldurf+name_urf[1]+'.csv')
        self.delta_y = np.array(df)

        df = pd.read_csv(self.foldurf+name_urf[2]+'.csv')
        self.isprocessed = np.array(df)

# Register two images for one task
class img2vxy:
    def __init__(self,ker_size,win_size,startxy,lengthxy,
                 img_fold_urf,img_old_urf,img_new_urf,
                 foldurf_fourier,finish_name_urf_fourier,err_name_urf_fourier,
                 foldurf_equal,finish_name_urf_equal,err_name_urf_equal,
                 ):
        self.foldurf_fourier = foldurf_fourier
        self.finish_name_urf_fourier = finish_name_urf_fourier
        self.err_name_urf_fourier = err_name_urf_fourier

        self.foldurf_equal = foldurf_equal
        self.finish_name_urf_equal = finish_name_urf_equal
        self.err_name_urf_equal = err_name_urf_equal

        self.imgs = matchimg(img_fold_urf,img_old_urf,img_new_urf)
        self.imgs.read_img()

        self.fourier = img2vxy_fourier(self.imgs,ker_size,win_size,startxy,lengthxy,
                                       foldurf_fourier,finish_name_urf_fourier,err_name_urf_fourier)

        self.equal = img2vxy_equal(self.imgs,ker_size,win_size,startxy,lengthxy,
                                       foldurf_equal,finish_name_urf_equal,err_name_urf_equal)

    # Fourier procession
    def process_fourier(self):
        if os.path.exists(self.foldurf_fourier+self.finish_name_urf_fourier[0]+'.csv'):
            print('Reading Fourier completed task...')
            if self.fourier.read_finished_vxy(self.finish_name_urf_fourier) != None:
                print('Reading Fourier completed task finished!')
                return 1
            else:
                return 0
        elif os.path.exists(self.foldurf_fourier+self.err_name_urf_fourier[0]+'.csv'):
            print('Reading Fourier temporary task...')
            state = self.fourier.completeprocess(True,self.err_name_urf_fourier)
            return state
        else:
            print('Starting Fourier task...')
            state = self.fourier.completeprocess()
            return state

    # secondary procession
    def process_equal(self,day,isautosave = True, is_used_fourier=False):
        if os.path.exists(self.foldurf_equal+self.finish_name_urf_equal[0]+'.csv'):
            print('Reading secondary completed task...')
            if self.equal.read_finished_vxy(self.finish_name_urf_equal) != None:
                print('Reading secondary completed task finished!')
                return 1
            else:
                return 0
        elif os.path.exists(self.foldurf_equal+self.err_name_urf_equal[0]+'.csv'):
            print('Reading secondary temporary task...')
            state = self.equal.completeprocess(isautosave=isautosave,is_after_err=True,past_arr_urf=self.err_name_urf_equal)
            return state
        else:
            print('Starting secondary task')
            self.equal.read_fourier_vxy(self.foldurf_fourier,self.finish_name_urf_fourier,is_used_fourier,self.fourier)
            self.equal.save_range_data([-0.5*day,3*day],[-0.5*day,3*day])
            state = self.equal.completeprocess(isautosave=isautosave)
            return state

# Registration for the whole tasks
class process_all:
    def __init__(self,basis_fold_urf,sta_name,file_name,picture_add,
                 ker_size,win_size,startxy,lengthxy):
        self.basis_fold_urf = basis_fold_urf
        self.sta_name = sta_name
        self.file_name = file_name
        self.picture_add = picture_add

        self.lengthxy = lengthxy
        self.ker_size = ker_size
        self.win_size = win_size
        self.startxy = startxy

        self.initialization()

    def initialization(self):
        if not os.path.exists(self.basis_fold_urf):
            os.makedirs(self.basis_fold_urf)
        subfolders = ['datae','dataf']
        for sub in subfolders:
            sub_path = os.path.join(self.basis_fold_urf, sub)
            if not os.path.exists(sub_path):
                os.makedirs(sub_path)
        file_name, file_ext = os.path.splitext(self.sta_name)
        csv_path = f"{file_name}.csv"
        if os.path.exists(csv_path):
            self.sta_name = csv_path
            df = pd.read_csv(self.sta_name)
            self.all_state_list = np.array(df)
            return None
        try:
            if file_ext.lower() in ['.xlsx', '.xls']:
                df = pd.read_excel(self.sta_name)
                self.all_state_list = np.array(df)
                df.to_csv(csv_path, index=False)
                self.sta_name = csv_path
            elif file_ext.lower() == '.txt':
                df = pd.read_csv(self.sta_name, sep='\t')
                self.all_state_list = np.array(df)
                df.to_csv(csv_path, index=False)
                self.sta_name = csv_path
            else:
                raise ValueError("Unsupported File Format!")
        except Exception:
            print(f"Please confirm if the **statistics** file format is correct.")

    def forward(self,type,isbreak=True,is_equal_autosave = True):
        if type == 'f':
            for ii in range(self.all_state_list.shape[0]-1):
                if self.all_state_list[ii,1]!=0:
                    continue
                else:

                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii+1,0])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_fourier()

                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 1] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue
        if type == 'e':
            for ii in range(self.all_state_list.shape[0]-1):
                if self.all_state_list[ii,2]!=0:
                    continue
                else:

                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii+1,0])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_equal(self.all_state_list[ii,3],isautosave=is_equal_autosave)

                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 2] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue
        if type=='fe':
            for ii in range(self.all_state_list.shape[0]-1):
                if self.all_state_list[ii,2]!=0:
                    continue
                elif self.all_state_list[ii,1]!=0:
                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii+1,0])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_equal(self.all_state_list[ii,3],isautosave=is_equal_autosave)

                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 2] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue
                else:

                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii+1,0])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_fourier()
                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 1] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue

                    state = test.process_equal(self.all_state_list[ii,3],isautosave=is_equal_autosave,is_used_fourier=True)
                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 2] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue

        if type=='dfe': #double
            for ii in range(self.all_state_list.shape[0]):
                if self.all_state_list[ii,4]!=0:
                    continue
                elif self.all_state_list[ii,3]!=0:
                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii,1])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_equal(self.all_state_list[ii,2],isautosave=is_equal_autosave)

                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 4] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue
                else:

                    v_last = str(self.all_state_list[ii,0])
                    v_end = str(self.all_state_list[ii,1])
                    v_str = v_last+'-'+v_end
                    print(f'Start processing: {v_str}')
                    finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
                    err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
                    finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
                    err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
                    test = img2vxy(self.ker_size,self.win_size,self.startxy,self.lengthxy,
                                   self.picture_add,v_last+'.jpg',v_end+'.jpg',
                                   self.basis_fold_urf+'dataf/',finish_name_urf_fourier,err_name_urf_fourier,
                                   self.basis_fold_urf+'datae/',finish_name_urf_equal,err_name_urf_equal)
                    state = test.process_fourier()
                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 3] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue

                    state = test.process_equal(self.all_state_list[ii,2],isautosave=is_equal_autosave,is_used_fourier=True)
                    if state:
                        df = pd.read_csv(self.sta_name)
                        df.iloc[ii, 4] = 1
                        df.to_csv(self.sta_name, index=False)
                        print(f'{v_str} finished!')
                    else:
                        print(f'{v_str} temporarily stopped!')
                        if isbreak:
                            break
                        else:
                            continue

    def print_v(self,v,cmin,cmax,title=None):
        x = range(v.shape[1])
        y = range(v.shape[0])
        allx,ally = np.meshgrid(x,y)
        fig,ax = plt.subplots()
        cr = ax.pcolormesh(allx,ally,v,cmap='coolwarm',vmin=cmin,vmax=cmax)
        fig.colorbar(cr,ax=ax)
        ax.invert_yaxis()
        if title != None:
            plt.title(title)
        plt.show()

    def which_v(self,type,number,state=0):
        v_last = str(self.all_state_list[number,0])
        v_end = str(self.all_state_list[number+1,0])
        v_str = v_last+'-'+v_end
        finish_name_urf_fourier = [self.file_name[0]+v_str+'_f',self.file_name[1]+v_str+'_f',self.file_name[2]+v_str+'_f']
        # err_name_urf_fourier = ['err_'+self.file_name[0]+v_str+'_f','err_'+self.file_name[1]+v_str+'_f','err_'+self.file_name[2]+v_str+'_f']
        finish_name_urf_equal = [self.file_name[0]+v_str+'_e',self.file_name[1]+v_str+'_e',self.file_name[2]+v_str+'_e']
        # err_name_urf_equal = ['err_'+self.file_name[0]+v_str+'_e','err_'+self.file_name[1]+v_str+'_e','err_'+self.file_name[2]+v_str+'_e']
        if type=='f':
            if state==0:
                df = pd.read_csv(self.basis_fold_urf+'dataf/'+finish_name_urf_fourier[0]+'.csv')
                self.print_v((np.array(df)-self.win_size//2+self.ker_size//2)/self.all_state_list[number,3],-3,3,v_str)
                df = pd.read_csv(self.basis_fold_urf+'dataf/'+finish_name_urf_fourier[1]+'.csv')
                self.print_v((np.array(df)-self.win_size//2+self.ker_size//2)/self.all_state_list[number,3],-3,3,v_str)
            elif state==1:
                df = pd.read_csv(self.basis_fold_urf+'dataf/'+finish_name_urf_fourier[0]+'.csv')
                self.print_v((np.array(df)-self.win_size//2+self.ker_size//2)/self.all_state_list[number,3],-3,3,v_str)
            else:
                df = pd.read_csv(self.basis_fold_urf+'dataf/'+finish_name_urf_fourier[1]+'.csv')
                self.print_v((np.array(df)-self.win_size//2+self.ker_size//2)/self.all_state_list[number,3],-3,3,v_str)

        if type=='e':
            if state==0:
                df = pd.read_csv(self.basis_fold_urf+'datae/'+finish_name_urf_equal[0]+'.csv')
                self.print_v((np.array(df))/self.all_state_list[number,3],-2.5,2.5,v_str)
                df = pd.read_csv(self.basis_fold_urf+'datae/'+finish_name_urf_equal[1]+'.csv',)
                self.print_v((np.array(df))/self.all_state_list[number,3],-2.5,2.5,v_str)
            elif state==1:
                df = pd.read_csv(self.basis_fold_urf+'datae/'+finish_name_urf_equal[0]+'.csv')
                self.print_v((np.array(df))/self.all_state_list[number,3],-2.5,2.5,v_str)
            else:
                df = pd.read_csv(self.basis_fold_urf+'datae/'+finish_name_urf_equal[1]+'.csv',)
                self.print_v((np.array(df))/self.all_state_list[number,3],-2.5,2.5,v_str)

In [4]:
'''
The coefficients of the class:
    basis_fold: storage of the all workflow data
    sta_name: statistics file
    file_name: prefix name of the results
    picture_add: address of the images
    ker_size: size of search window
    win_size: size of search range
    startxy: topleft point of the result
    lengthxy: size of results
'''

ker_size = 45
win_size = 201
startij = [300,300]
lengthij = [3000,3000]

basis_fold = 'test\\'
picture_add = 'Pictures\jpg\\'
sta_name = 'statistics.xlsx'
file_name = ['vx','vy','isprocessed']

In [ ]:
start = process_all(basis_fold,sta_name,file_name,picture_add,ker_size,win_size,startij,lengthij)
start.forward('dfe',True,False)

'''
The coefficients of the forward function:
1, type:
    'f': only Fourier procession, with statistics column: ['date_{i}', 'date_{i+1}-date_{i}', isFourier, isSecondary]
    'e': only secondary procession, with statistics column: ['date_{i}', 'date_{i+1}-date_{i}', isFourier, isSecondary]
    'fe': Fourier and secondary procession, with statistics column: ['date_{i}', 'date_{i+1}-date_{i}', isFourier, isSecondary]
    'dfe': Fourier and secondary procession, with statistics column: ['date1', 'date2', 'date2-date1', isFourier, isSecondary]
2, isbreak:
    True: Stop the program after interruption
    False: Perform the next registration (if there is) after interruption
3, is_equal_autosave:
    True: Perform automatic saving in secondary processing
    False: Do not perform automatic saving in secondary processing
'''

Start processing: 20180401-20180417
Reading secondary temporary task...


  0%|          | 0/1500 [00:00<?, ?it/s]C:\Users\1\AppData\Local\Temp\ipykernel_24468\3666629539.py:390: RuntimeWarning: invalid value encountered in divide
  tmp = tmp/np.abs(tmp)
 43%|████▎     | 643/1500 [11:56<15:15,  1.07s/it]  